In [36]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
from utils.dataset import *
from utils.train import model_setting
from utils.model import *
from torch.utils.data import DataLoader,random_split
from torchvision.transforms import v2 as transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from torch import nn

class HungarianLoss(nn.Module):
    def __init__(self, num_classes, weight_dict={'cls': 1, 'bbox': 5}, eos_coef=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.weight_dict = weight_dict
        self.eos_coef = eos_coef
        self.bbox_criterion = nn.L1Loss(reduction='none')
        self.cls_criterion = nn.CrossEntropyLoss(reduction='none')

    def forward(self, outputs, targets):
        bs, num_queries = outputs['pred_logits'].shape[:2]
        out_prob = outputs['pred_logits'].flatten(0, 1).softmax(-1)  # [bs*num_queries, num_classes]
        out_bbox = outputs['pred_boxes'].flatten(0, 1)  # [bs*num_queries, 4]

        tgt_ids = torch.cat([t['labels'] for t in targets])  # [sum(num_objs)]
        tgt_bbox = torch.cat([t['boxes'] for t in targets])  # [sum(num_objs), 4]

        # Compute the cost matrix
        cost_class = -out_prob[:, tgt_ids]
        cost_bbox = self.bbox_criterion(out_bbox, tgt_bbox)
        cost_bbox = cost_bbox.sum(-1)
        cost_matrix = self.weight_dict['cls'] * cost_class + self.weight_dict['bbox'] * cost_bbox
        cost_matrix = cost_matrix.view(bs, num_queries, -1).cpu()

        # Compute the optimal assignment
        indices = [linear_sum_assignment(c[i]) for i, c in enumerate(cost_matrix.split(1, -1))]
        indices = [(torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64)) for i, j in indices]
        indices = [(i.to(out_prob.device), j.to(out_prob.device)) for i, j in indices]

        # Compute the losses
        cls_loss = self.cls_criterion(out_prob, tgt_ids)
        bbox_loss = self.bbox_criterion(out_bbox, tgt_bbox)
        cls_loss = sum([cls_loss[i, j].mean() for (i, j) in indices])
        bbox_loss = sum([bbox_loss[i, j].mean() for (i, j) in indices])
        loss = cls_loss + bbox_loss

        return loss


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
dataset = Detection_data('D:\Folder\Vscode\Git\Airflow organization prediction\data\detection_data')

In [39]:
train_data,val_data,test_data = random_split(dataset,[0.8,0.1,0.1])
train_loader = DataLoader(train_data,batch_size=2,shuffle=True)
val_loader = DataLoader(val_data,batch_size=2,shuffle=False)

In [38]:
model = DETR(num_classes=2, hidden_dim=256, n_heads=8, num_encoder_layers=6, num_decoder_layers=6)
criterion = HungarianLoss(2)
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)

In [ ]:
for epoch in range(1):
    for inputs,